# Clean Network

Cleans the network files to be only what is specified in the data model so as not to store extraneous data / minimize space.

## Roadway

In [39]:
from pathlib import Path

from network_wrangler import load_roadway_from_dir, load_transit, write_roadway, write_transit
from network_wrangler.models.gtfs.tables import (
    WranglerFrequenciesTable,
    WranglerShapesTable,
    WranglerStopsTable,
    WranglerStopTimesTable,
    WranglerTripsTable,
)
from network_wrangler.models.roadway.tables import RoadLinksTable, RoadNodesTable, RoadShapesTable


In [ ]:
def shared_fields_w_datamodel(df, model):
    """Returns a list of fields that are in both the dataframe and the datamodel."""
    model_fields = list(model.__fields__.keys())
    df_fields = list(df.columns)
    return list(set(model_fields).intersection(df_fields))

In [2]:
og_net = load_roadway_from_dir(Path.cwd())

Skipping field outboundReferenceIds: unsupported OGR type: 5
Skipping field inboundReferenceIds: unsupported OGR type: 5


In [22]:
links_data_model_fields = list(RoadLinksTable.__fields__.keys())
shared_fields = set(links_data_model_fields).intersection(set(og_net.links_df.columns))
DEL_FIELDS = ['locationReferences']
KEEP_FIELDS = ['geometry']
specified_fields = ["model_link_id", "A", "B", "name", "osm_link_id"]
remaining_fields = list(shared_fields.union(set(KEEP_FIELDS)) - set(DEL_FIELDS) - set(specified_fields))
final_fields = specified_fields + remaining_fields
og_net.links_df = og_net.links_df[final_fields]

In [27]:
nodes_data_model_fields = list(RoadNodesTable.__fields__.keys())
shared_node_fields = list(set(nodes_data_model_fields).intersection(set(og_net.nodes_df.columns)))
og_net.nodes_df = og_net.nodes_df[shared_node_fields]

In [32]:
shapes_data_model_fields = list(RoadShapesTable.__fields__.keys())
shared_shapes_fields = list(set(shapes_data_model_fields).intersection(set(og_net.shapes_df.columns)))
og_net.shapes_df = og_net.shapes_df[shared_shapes_fields]

In [36]:
write_roadway(og_net, Path.cwd()/"smaller")

In [37]:
smaller_net = load_roadway_from_dir(Path.cwd()/"smaller")
smaller_net.links_df

## Transit

In [40]:
og_transit = load_transit(Path.cwd())

/opt/miniconda3/envs/wrangler-dev/lib/python3.10/site-packages/pandera/engines/pandas_engine.py:898: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  col = to_datetime_fn(col, **self.to_datetime_kwargs)
/opt/miniconda3/envs/wrangler-dev/lib/python3.10/site-packages/pandera/engines/pandas_engine.py:898: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  col = to_datetime_fn(col, **self.to_datetime_kwargs)


In [42]:
og_transit.feed.trips

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,trip_destination,trip_short_name,projects,trip_route_name,pub_dir_id,rm_block_id
0,452-111,JUN19-MVS-BUS-Weekday-01,14940701-JUN19-MVS-BUS-Weekday-01,Westbound 452 Express / Minneapolis,1,1874,4520004,1,None,None,,NaN,None,None
1,3-111,JUN19-MVS-BUS-Weekday-01,14941148-JUN19-MVS-BUS-Weekday-01,Eastbound 3B Como Av/St Paul - Depot/Via Front,0,1018,30009,1,None,None,,NaN,None,None
2,3-111,JUN19-MVS-BUS-Weekday-01,14941151-JUN19-MVS-BUS-Weekday-01,Westbound 3B Como-Front/Dwtn Mpls/Target Field,1,1017,30014,1,None,None,,NaN,None,None
3,3-111,JUN19-MVS-BUS-Weekday-01,14941153-JUN19-MVS-BUS-Weekday-01,Westbound 3B Como-Front/Dwtn Mpls/Target Field,1,1017,30014,1,None,None,,NaN,None,None
4,3-111,JUN19-MVS-BUS-Weekday-01,14941163-JUN19-MVS-BUS-Weekday-01,Eastbound 3B Como Av/St Paul - Depot/Via Front,0,1036,30009,1,None,None,,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,902-111,JUN19-RAIL-Weekday-01,14969944-JUN19-RAIL-Weekday-01,Eastbound to St. Paul-Union Depot,0,52,9020001,1,None,None,,NaN,None,None
58,902-111,JUN19-RAIL-Weekday-01,14969962-JUN19-RAIL-Weekday-01,Eastbound to St. Paul-Union Depot,0,53,9020001,1,None,None,,NaN,None,None
59,484-71,20190216-Weekday-04,121-RL-484.0N-0632-20190216-Weekday-04,484 ST PAUL,1,121,4840095,0,ST PAUL,484.0,,484.0,Northbound,121-RL
60,480-71,20190216-Weekday-04,031-CT-480.1N-0601-20190216-Weekday-04,480 ST PAUL,1,31,4800212,0,ST PAUL,480.0,,480.0,Northbound,031-CT
